## Xarray engine: size-1 dimension as a variable attribute

First, we get some GRIB forecast data on single levels and read it into a GRIB fieldlist.

In [1]:
import earthkit.data as ekd

In [2]:
ds_fl = ekd.from_source("sample", "aifs-sfc.grib")

Examine the metadata content of the field list (for readibility, we select a single forecast start time and step).

In [3]:
ds_fl.sel(dataDate=20251212, dataTime=0, step=0).ls()

,centre,shortName,typeOfLevel,level,dataDate,dataTime,stepRange,dataType,number,gridType
0,ecmf,tcw,entireAtmosphere,0,20251212,0,0,fc,None,regular_ll
1,ecmf,msl,meanSea,0,20251212,0,0,fc,None,regular_ll
2,ecmf,10u,heightAboveGround,10,20251212,0,0,fc,None,regular_ll
3,ecmf,2t,heightAboveGround,2,20251212,0,0,fc,None,regular_ll
4,ecmf,tp,surface,0,20251212,0,0,fc,None,regular_ll


We see that each variable has a different level and/or type of level. This would be an obstacle in forming an xarray Dataset object with an explicit level dimension (in any of the form provided by ``level_dim_mode``). However, we can use an option
``dims_as_attrs`` to turn size-1 level dimensions into variables' attributes, instead of just squeezing them.

Below, we use the default ``level_dim_mode="level"`` which builds ``"level"`` and ``"level_type"`` dimensions, and therefore these two dimensions are turned into attributes.

In [4]:
ds = ds_fl.to_xarray(
    profile=None, 
    dims_as_attrs=["level", "level_type"], 
    add_earthkit_attrs=False, 
)
ds

<xarray.Dataset> Size: 14kB
Dimensions:                  (forecast_reference_time: 2, step: 2, latitude: 7,
                              longitude: 12)
Coordinates:
  * forecast_reference_time  (forecast_reference_time) datetime64[ns] 16B 202...
  * step                     (step) timedelta64[ns] 16B 00:00:00 06:00:00
  * latitude                 (latitude) float64 56B 90.0 60.0 ... -60.0 -90.0
  * longitude                (longitude) float64 96B 0.0 30.0 ... 300.0 330.0
Data variables:
    10u                      (forecast_reference_time, step, latitude, longitude) float64 3kB ...
    2t                       (forecast_reference_time, step, latitude, longitude) float64 3kB ...
    msl                      (forecast_reference_time, step, latitude, longitude) float64 3kB ...
    tcw                      (forecast_reference_time, step, latitude, longitude) float64 3kB ...
    tp                       (forecast_reference_time, step, latitude, longitude) float64 3kB ...

Inspection of the variables' attributes give this:

In [5]:
{v: ds[v].attrs for v in ds}

{'10u': {'level': 10, 'level_type': 'heightAboveGround'},
 '2t': {'level': 2, 'level_type': 'heightAboveGround'},
 'msl': {'level': 0, 'level_type': 'meanSea'},
 'tcw': {'level': 0, 'level_type': 'entireAtmosphere'},
 'tp': {'level': 0, 'level_type': 'surface'}}

Using ``level_dim_mode="level_per_type"`` we can declare the **template** dimension ``"<level_per_type>"`` to be turned into variables' attributes. This transforms the actual dimensions like ``"surface", "heightAboveGround"``, etc. to variables' attributes, since all of them are of size 1.

In [6]:
ds2 = ds_fl.to_xarray(
    profile=None, 
    level_dim_mode="level_per_type", 
    dims_as_attrs="<level_per_type>", 
    add_earthkit_attrs=False, 
)
ds2

<xarray.Dataset> Size: 14kB
Dimensions:                  (forecast_reference_time: 2, step: 2, latitude: 7,
                              longitude: 12)
Coordinates:
  * forecast_reference_time  (forecast_reference_time) datetime64[ns] 16B 202...
  * step                     (step) timedelta64[ns] 16B 00:00:00 06:00:00
  * latitude                 (latitude) float64 56B 90.0 60.0 ... -60.0 -90.0
  * longitude                (longitude) float64 96B 0.0 30.0 ... 300.0 330.0
Data variables:
    10u                      (forecast_reference_time, step, latitude, longitude) float64 3kB ...
    2t                       (forecast_reference_time, step, latitude, longitude) float64 3kB ...
    msl                      (forecast_reference_time, step, latitude, longitude) float64 3kB ...
    tcw                      (forecast_reference_time, step, latitude, longitude) float64 3kB ...
    tp                       (forecast_reference_time, step, latitude, longitude) float64 3kB ...

In [7]:
{v: ds2[v].attrs for v in ds2}

{'10u': {'heightAboveGround': 10},
 '2t': {'heightAboveGround': 2},
 'msl': {'meanSea': 0},
 'tcw': {'entireAtmosphere': 0},
 'tp': {'surface': 0}}

### More elaborate example

Similarly, we can deal with a dataset, in which some of variables are single-level, and others are multi-level (however, with consistent vertical coordinates).

In [8]:
ds_fl2 = ekd.from_source("sample", "aifs-pl_sfc.grib")

In [9]:
ds_fl2.sel(dataDate=20251212, dataTime=0, step=0).ls()

,centre,shortName,typeOfLevel,level,dataDate,dataTime,stepRange,dataType,number,gridType
0,ecmf,z,isobaricInhPa,500,20251212,0,0,fc,None,regular_ll
1,ecmf,z,isobaricInhPa,850,20251212,0,0,fc,None,regular_ll
2,ecmf,z,isobaricInhPa,1000,20251212,0,0,fc,None,regular_ll
3,ecmf,tcw,entireAtmosphere,0,20251212,0,0,fc,None,regular_ll
4,ecmf,msl,meanSea,0,20251212,0,0,fc,None,regular_ll
5,ecmf,10u,heightAboveGround,10,20251212,0,0,fc,None,regular_ll
6,ecmf,2t,heightAboveGround,2,20251212,0,0,fc,None,regular_ll
7,ecmf,tp,surface,0,20251212,0,0,fc,None,regular_ll


In [10]:
ds3 = ds_fl2.to_xarray(
    profile=None, 
    level_dim_mode="level_per_type", 
    dims_as_attrs=["<level_per_type>"], 
    add_earthkit_attrs=False, 
)
ds3

<xarray.Dataset> Size: 22kB
Dimensions:                  (forecast_reference_time: 2, step: 2, latitude: 7,
                              longitude: 12, isobaricInhPa: 3)
Coordinates:
  * forecast_reference_time  (forecast_reference_time) datetime64[ns] 16B 202...
  * step                     (step) timedelta64[ns] 16B 00:00:00 06:00:00
  * isobaricInhPa            (isobaricInhPa) int64 24B 500 850 1000
  * latitude                 (latitude) float64 56B 90.0 60.0 ... -60.0 -90.0
  * longitude                (longitude) float64 96B 0.0 30.0 ... 300.0 330.0
Data variables:
    10u                      (forecast_reference_time, step, latitude, longitude) float64 3kB ...
    2t                       (forecast_reference_time, step, latitude, longitude) float64 3kB ...
    msl                      (forecast_reference_time, step, latitude, longitude) float64 3kB ...
    tcw                      (forecast_reference_time, step, latitude, longitude) float64 3kB ...
    tp                       (forecast_reference_time, step, latitude, longitude) float64 3kB ...
    z                        (forecast_reference_time, step, isobaricInhPa, latitude, longitude) float64 8kB ...

In this example the all the level-like dimensions were transformed into variables' attributes except for the dimension ``"isobaricInhPa"`` of the variable ``"z"``, since it is of size 3.

In [11]:
{v: ds3[v].attrs for v in ds3}

{'10u': {'heightAboveGround': 10},
 '2t': {'heightAboveGround': 2},
 'msl': {'meanSea': 0},
 'tcw': {'entireAtmosphere': 0},
 'tp': {'surface': 0},
 'z': {}}

It works similarly for the default ``level_dim_mode="level"``:

In [12]:
ds4 = ds_fl2.to_xarray(
    profile=None, 
    dims_as_attrs=["level", "level_type"], 
    add_earthkit_attrs=False, 
)
ds4

<xarray.Dataset> Size: 22kB
Dimensions:                  (forecast_reference_time: 2, step: 2, latitude: 7,
                              longitude: 12, level: 3)
Coordinates:
  * forecast_reference_time  (forecast_reference_time) datetime64[ns] 16B 202...
  * step                     (step) timedelta64[ns] 16B 00:00:00 06:00:00
  * level                    (level) int64 24B 500 850 1000
  * latitude                 (latitude) float64 56B 90.0 60.0 ... -60.0 -90.0
  * longitude                (longitude) float64 96B 0.0 30.0 ... 300.0 330.0
Data variables:
    10u                      (forecast_reference_time, step, latitude, longitude) float64 3kB ...
    2t                       (forecast_reference_time, step, latitude, longitude) float64 3kB ...
    msl                      (forecast_reference_time, step, latitude, longitude) float64 3kB ...
    tcw                      (forecast_reference_time, step, latitude, longitude) float64 3kB ...
    tp                       (forecast_reference_time, step, latitude, longitude) float64 3kB ...
    z                        (forecast_reference_time, step, level, latitude, longitude) float64 8kB ...

In [13]:
{v: ds4[v].attrs for v in ds4}

{'10u': {'level': 10, 'level_type': 'heightAboveGround'},
 '2t': {'level': 2, 'level_type': 'heightAboveGround'},
 'msl': {'level': 0, 'level_type': 'meanSea'},
 'tcw': {'level': 0, 'level_type': 'entireAtmosphere'},
 'tp': {'level': 0, 'level_type': 'surface'},
 'z': {'level_type': 'isobaricInhPa'}}